In [1]:
%%capture
!pip install transformers==3.5.1

In [2]:
!pip install torch==1.4.0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 7.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0+cu113 requires torch==1.12.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.4.0 which is incompatible.
fastai 2.7.6 requires torch<1.13,>=1.7, but you have torch 1.4.0 which is incompatible.


Hugging face is an organization that is on the path of democratizing AI through natural language processing. Their open source transformers library is very popu;ar among the Natural Language processing community. It is very useful and powerful for several NLP and natural language Understanding (NLU) tasks. It includes thousands of pre-trained models in more than 100 languages/ One of the many advantages of the transformer library is that it is compatible with both PyTorch and Tensorflow

# BERT embeddings.


In [3]:
from transformers import BertModel, BertTokenizer
import torch

In [4]:
# we need to download BERT pretrained model
model= BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
# Next we will ned to download and load the tokenizer that was used to pre-train the bert-base-uncased
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
# Define the input
sentence = 'I love VietNam'
# We need tokenize the sentence and obtains the tokens
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'vietnam']


In [7]:
# Now we will add the [CLS] token at the beginning
# And we will add the [SEP] token at the end of the token list
tokens = ['[CLS]'] + tokens+['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'vietnam', '[SEP]']


As can be seen from the above resilt, we have a [CLS] token at the beginning and an [SEP] token at the end of our tokens list. We can also see that the length of our tokens list is 5.

If we want to increase the lenght of our tokens list to 7

In [8]:
tokens= tokens +['[ROL]']+['[ROL]']
print(tokens)

['[CLS]', 'i', 'love', 'vietnam', '[SEP]', '[ROL]', '[ROL]']


As can be seen from the above list, we have the tokens list with [ROL] toekns and the length of our token list is increased to 7.

The next step is we need to create the attention mask. We need to set the attention mask value to 1 if the token is not [ROL] token, else we set the token mast to 0.

In [9]:
attention_mask = [1 if i!='[ROL]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


As can be seen from the above result, we have the attention mask values 0 at positions where have a [ROL] token and 1 at the other position

In [10]:
# Convert all the toeks to their token IDS
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1045, 2293, 5148, 102, 100, 100]


As can be seen from the above result, each toek is mapped to a unique token  ID.

In [11]:
# convert token_ids and attention_mask to tensors
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [12]:
# We wneed to feed both token_ids and attention_mask to the pre-trained BERT model and get the embedding.

# Getting the embedding
model will return the output as a tuple with 2 values. The first value indicates the hidden state represnetation, hidden_rep and it contains the representation of all the tokens obtained from the final encode (encoder 12) and the second value, cls_head which contains the representation of [ROL] token

In [13]:
hidden_rep, cls_head = model(token_ids, attention_mask= attention_mask)

In [14]:
print(hidden_rep)

tensor([[[-0.0962,  0.2936, -0.3077,  ..., -0.2777,  0.2598,  0.4106],
         [ 0.1180,  0.5573,  0.0096,  ..., -0.3113,  0.5982, -0.1665],
         [ 0.9262,  0.9651,  0.6678,  ..., -0.1362,  0.5367, -0.2279],
         ...,
         [ 0.7797,  0.2199, -0.1623,  ..., -0.1704, -0.5798, -0.3032],
         [-0.2176,  0.2169,  0.4124,  ...,  0.3516, -0.2273, -0.1362],
         [-0.1024,  0.0676,  0.4749,  ...,  0.4331, -0.2904, -0.0798]]],
       grad_fn=<NativeLayerNormBackward>)


In [15]:
print(hidden_rep.shape)

torch.Size([1, 7, 768])


The size [1, 7, 768] indicates [batch_size, sequence_length, hidden_size]


The batch_size is 1. The sequence length is the token legnth. Because we have 7 tokens, the sequence lenght is 7. The hiddne size is the representation (embedding) size and it is 768 for the BERT-base model.

We can get the repersentation of each token:


*   hidden_rep[0][0] gives the representation of the first token which is [ROL]
*    hidden_rep[0][1] gives the representation of the second token which is I

*   hidden_rep[0][2] gives the representation of the second token which is love

In this way we can obtain the contextual representation of all tokens. This means the contextualized word embeddings of all the words in the given sentence.





In [16]:
print(cls_head.shape)

torch.Size([1, 768])


[1, 768] shows [batch_size, hidden_size]
cls_head holds the aggregate representation of the sentence so we cna use cls_head as the representation fo the sentence I love VietNam

# Extracting the embeddings

We need to downloa dhte pre-trained BERT model and tokenizer. we need to set the output_hidden_states = True and setting this to True helps us to obtain embedding from all the encoder layers

In [17]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [18]:
# Define the input
sentence = 'I love VietNam'
# We need tokenize the sentence and obtains the tokens
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'vietnam']


In [19]:
# Now we will add the [CLS] token at the beginning
# And we will add the [SEP] token at the end of the token list
tokens = ['[CLS]'] + tokens+['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'vietnam', '[SEP]']


In [20]:
tokens= tokens +['[ROL]']+['[ROL]']
print(tokens)

['[CLS]', 'i', 'love', 'vietnam', '[SEP]', '[ROL]', '[ROL]']


In [21]:
attention_mask = [1 if i!='[ROL]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


In [22]:
# Convert all the toeks to their token IDS
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1045, 2293, 5148, 102, 100, 100]


In [23]:
# convert token_ids and attention_mask to tensors
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

# Getting the embeddings.
Because we set the output_hidden_states= True while defining the model to get the embeddings from all the encoder layers, now the model will return an ouput tupe with 3 values

In [24]:
last_hidden_state, pooler_output, hidden_states = model(token_ids, attention_mask = attention_mask)

As can be seen from the above code, the first value, last_hidden_state contains the representation of all the tokens obtained only from the final encoder layer(encoder 12).

Next is the pooler_output shows the representation of the [CLS] token from the final encoder layer which is further processd by a linear and tanh activation function.
hidden_states contains the representation of all the tokens obtained from all the encoder layers.

The next step is to look tat each of these values and understand them in more detail.

First let's take a look at last_hidden_state. As we did learn it holds the representation of all the tokens obtained only from the final encoder layer (encoder 12)

In [25]:
# It is a time to print the shape of last_hidden_state
last_hidden_state.shape

torch.Size([1, 7, 768])

The size [1, 7, 768] indicates [batch_size, sequence_length, hidden_size]

The batch_size is 1. The sequence length is the token legnth. Because we have 7 tokens, the sequence lenght is 7. The hiddne size is the representation (embedding) size and it is 768 for the BERT-base model.

We can get the repersentation of each token:

hidden_rep[0][0] gives the representation of the first token which is [ROL]

hidden_rep[0][1] gives the representation of the second token which is I

hidden_rep[0][2] gives the representation of the second token which is love

Similarly, we can obtain the representation of all the tokens from the final encoder layer.

Next we have a pooler_output, which contains the representation of the [ROL] token from the final encoder layer which is further processed by a linear and tanh activation function.

In [26]:
pooler_output.shape

torch.Size([1, 768])

[1, 768] shows [batch_size, hidden_size] cls_head holds the aggregate representation of the sentence so we cna use cls_head as the representation fo the sentence I love VietNam.

And then finally we have the hidden_states, which contains the representation of all the tokens obtained from all the encoder layers. It is a tuple containing 13 values holding the representation of all encoder layers (hidden layers), from the input embedding layer h0 to the final encoder layer h12


In [27]:
len(hidden_states)

13

As can be seen from the above result, it contains 13 values holding the representation of all layers

hidden_states[0] contains the representation of all the tokens obtained from the input embedding layer h0


hidden_state[1] contains the representation of all the tokens obtained from the first encoder layer h1


hidden_state[2] contains the representation of all the tokens obtained from the first encoder layer h2


hidden_state[12] contains the representation of all the tokens obtained from the first encoder layer h12

Now let's explore this more.

In [28]:
# print the shape of hdden_states[0] which contains the representation of all the tokens obtained from the input embedding layer h0
hidden_states[0].shape

torch.Size([1, 7, 768])

The size [1, 7, 768] indicates [batch_size, sequence_length, hidden_size]

In [29]:
hidden_states[1].shape


torch.Size([1, 7, 768])

Therefore, in this way we can obtain the embedding of tokens from all the encoder layers. 